In [ ]:
import os
import sys
from pathlib import Path

# プロジェクトのルートディレクトリを取得
project_root = Path(os.getcwd()).resolve().parent # 必要に応じて調整
sys.path.append(str(project_root))

In [ ]:
import numpy as np
import pandas as pd
from openai import AzureOpenAI
import glob
from tqdm import tqdm
import pickle
import zipfile
import io
import base64
from PIL import Image
import IPython.display
from IPython.display import display, Markdown
import subprocess
import configparser
from pdf2image import convert_from_path
import faiss

# from google.colab import userdata
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

## 設定

### pathの設定

In [ ]:
base_path = ".."
# 出力フォルダがなければ作成
for dir_name in ["process", "output", "models"]:
    if not os.path.exists(f"{base_path}/{dir_name}"):
        os.makedirs(f"{base_path}/{dir_name}")

# 入力フォルダと出力フォルダ
input_folder = "../documents/"
output_folder = "../documents_md/"

# 出力フォルダがなければ作成
os.makedirs(output_folder, exist_ok=True)

# PDF ファイルを取得
# 動作確認用に1ファイルに絞っている
pdf_files = glob.glob(os.path.join(input_folder, "*.pdf"))
pdf_files

In [ ]:
class RAGConfig:
    """
    設定ファイル(config.ini)を読み込むクラス。
    configファイルは下記のように構成されます。
    ```
    [DEFAULT]
    URL=<url>
    API_KEY=<api_key>
    API_VERSION=<api_version>
    CHAT_MODEL=<CHAT_MODEL>
    EMBEDDING_MODEL=<EMBEDDING_MODEL>
    ```
    """

    def __init__(self, config_path):
        self.config = self._load_config(config_path)

    def _load_config(self, config_path):
        """
        設定ファイルを読み込み、DEFAULTセクションを取得する。

        Args:
            config_path (str): 設定ファイルのパス。

        Returns:
            ConfigParser.SectionProxy: DEFAULTセクションの設定。
        """
        config_ini = configparser.ConfigParser()
        config_ini.read(config_path, encoding="utf-8")
        return config_ini["DEFAULT"]

    @property
    def azure_openai_endpoint(self):
        """Azure OpenAIエンドポイントURLを取得"""
        return self.config["URL"]

    @property
    def azure_openai_api_key(self):
        """Azure OpenAIのAPIキーを取得"""
        return self.config["API_KEY"]

    @property
    def api_version(self):
        """APIバージョンを取得"""
        return self.config["API_VERSION"]

    @property
    def CHAT_MODEL(self):
        """使用するチャットモデル名を取得"""
        return self.config["CHAT_MODEL"]

    @property
    def EMBEDDING_MODEL(self):
        """使用する埋め込みモデル名を取得"""
        return self.config["EMBEDDING_MODEL"]


In [ ]:
# 設定を読み込む
config = RAGConfig('../config/config.ini')

# Azure OpenAI クライアントを初期化（AzureOpenAI は別途定義されていることを前提）
client = AzureOpenAI(
    api_key=config.azure_openai_api_key,
    azure_endpoint=config.azure_openai_endpoint,
    api_version=config.api_version
)

## vector_store作成

### OCR

In [ ]:
# 各 PDF に `yomitoku` を適用
for pdf_file in pdf_files:
    file = os.path.basename(pdf_file).split(".")[0]
    print(f"Processing: {pdf_file}")
    command = f'yomitoku "{pdf_file}" -f md -o {output_folder}{file}'
    subprocess.run(command, shell=True)

print("すべての PDF を Markdown に変換しました。")

In [ ]:
# 企業名付与
concept_dicts = {
    '../documents/9.pdf': 'ハウス食品グループ',
     '../documents/8.pdf': 'サントリーグループ',
     '../documents/16.pdf': '東急不動産ホールディングス',
     '../documents/17.pdf': 'TOYOエンジニアリング株式会社',
     '../documents/15.pdf': '全国保証株式会社',
     '../documents/14.pdf': '髙松コンストラクショングループ',
     '../documents/10.pdf': 'パナソニック',
     '../documents/11.pdf': '株式会社メディアドゥ',
     '../documents/13.pdf': 'ライフコーポレーション',
     '../documents/12.pdf': 'モスグループ',
     '../documents/19.pdf': '明治ホールディングス',
     '../documents/18.pdf': '日清食品ホールディングス',
     '../documents/6.pdf': 'クレハ',
     '../documents/7.pdf': 'グローリー株式会社',
     '../documents/5.pdf': 'キッツ株式会社',
     '../documents/4.pdf': 'カゴメ株式会社',
     '../documents/1.pdf': '株式会社4℃ホールディングス',
     '../documents/3.pdf': '日産自動車',
     '../documents/2.pdf': 'IHIグループ'}

In [ ]:
text_data = []
for pdf_file in tqdm(pdf_files):
    file = os.path.basename(pdf_file).split(".")[0]     
    images = convert_from_path(pdf_file) 
    concept = concept_dicts[pdf_file]
    for page, image in enumerate(images, start=1):
        image_bytes = io.BytesIO()
        image.save(image_bytes, format="JPEG")
        encode_image = base64.b64encode(image_bytes.getvalue()).decode('utf-8')    

        md_path = f"{output_folder}{file}/documents_{file}_p{page}.md"
        with open(md_path, "r", encoding="utf-8") as f:
            md_content = f.read()
        text_data.append({
            "pdf_path":pdf_file,
            "page": page,
            "concept": concept,
            "search": f"# {concept}\n{md_content}",
            "image": encode_image
        })

rag_df = pd.DataFrame(text_data)
rag_df.to_pickle("../process/rag_df.pkl")

## ベクトル化

In [ ]:
class CreateVectorStore:
    """
    """
    def __init__(self, client, config):
        self.client = client  # Azure OpenAI クライアント
        self.config = config  # チャットモデル名

    def encode_text(self, text):
        """
        テキストを埋め込みベクトルに変換する。

        Args:
            text (str): 埋め込み対象のテキスト。

        Returns:
            np.array: 埋め込みベクトル。
        """
        vector = (
            self.client.embeddings.create(
                input=text,
                model=self.config.EMBEDDING_MODEL,
            )
            .data[0]
            .embedding
        )
        return vector

    def encode_questions(self, df):
        """
        DataFrame内の質問テキストを埋め込みベクトルに変換する。

        Args:
            df (DataFrame): 'search'列に質問を含むDataFrame。

        Returns:
            np.ndarray: 質問の埋め込みベクトルの配列。
        """
        embeddings = []
        for question in tqdm(df["search"]):
            embedding = self.encode_text(question)
            embeddings.append(embedding)
        return np.vstack(embeddings)

    def build_faiss_index(self, embeddings):
        """
        埋め込みベクトルを使用してFAISSインデックスを構築する。

        Args:
            embeddings (np.ndarray): 埋め込みベクトルの配列。

        Returns:
            faiss.IndexFlatL2: 構築されたFAISSインデックス。
        """
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index.add(embeddings)
        return index

In [ ]:
# RAGProcessor を初期化
create_vector_store = CreateVectorStore(
    client=client,
    config=config
)

# 質問の埋め込みベクトルを作成
question_embeddings = create_vector_store.encode_questions(rag_df)

# FAISSインデックスを構築
index = create_vector_store.build_faiss_index(question_embeddings)

In [ ]:
rag_df["context"] = rag_df["search"]
# FAISSモデルをファイルに保存
faiss_model = {
    "index": index,
    "rag_df": rag_df,
}

with open(f"{base_path}/models/faiss_model.pkl", "wb") as f:
    pickle.dump(faiss_model, f)  

## 文章生成

In [ ]:
with open(f"{base_path}/models/faiss_model.pkl", "rb") as f:
    faiss_model = pickle.load(f)
index = faiss_model["index"]
rag_df = faiss_model["rag_df"]

In [ ]:
# 動作確認用に検証用に5件に絞っている
query_df = pd.read_csv(f"{base_path}/input/query.csv").head(5)

In [ ]:
class RAG_Generater():
    def __init__(self, client, config, index):
        self.client = client  # Azure OpenAI クライアント
        self.config = config  # チャットモデル名
        self.index = index
        
    def extract_rag_indices(self, question, k=2):
        query_embedding = (
            self.client.embeddings.create(
                input=question,
                model=self.config.EMBEDDING_MODEL,
            )
            .data[0]
            .embedding
        )
        distances, indices = self.index.search(
            np.array([query_embedding], dtype=np.float32), k=k
        )
        return indices[0]
    
    def generate_process_answer(
            self,
            question,
            text=None,
            max_tokens=1000):

        text_prefix = f"""
        次のテキストは関連したドキュメントをMarkdownに変換したものである。
        これを元に、質問に対して**計算過程を明確に説明し、最後に答えを出せ**。

        - **数値を求める場合**
          1. 必要な数値をすべてリストアップする。
          2. 計算式を明示し、途中計算をすべて書く。必ず下記に記載の計算規則に従うこと。
          3. **計算過程をステップごとに示し、答えを最後に書く**。
          4. **もう一度質問と計算規則をかきだし、照らし合わせて問われていることが間違っていないか確認し、誤りを防ぐ**。
          もし質問に正確に答えることができない、または推測が入っている場合は**「分かりません」**と答えること。
        - **計算規則**
            - 小数第n位を四捨五入する場合は第n位で四捨五入を行い、答えは小数第(n-1)桁とすること。
            - 最大値を求める場合は、リストアップした中から上位を算出し、その中で最大のものを求めること。
        - **情報が不足している場合**
          - **「分かりません」** と明確に回答し、足りない情報を指摘する""",
        content = [
            {
                "type": "text",
                "text": f"""
                {text_prefix}
                {question}
                参考：
                {text}
                """
            }
        ]

        response = self.client.chat.completions.create(
            model=self.config.CHAT_MODEL,
            messages=[
                {
                    "role":"user",
                    "content":content
                }
            ],
            max_tokens=max_tokens,
            temperature=0,        
        )
        answer = response.choices[0].message.content
        if answer is None or answer.strip() == "":
            answer = "分かりません"
        answer = answer.strip()
        return answer

    def generate_answer(self, question, process_answer, max_tokens=54):
        response = self.client.chat.completions.create(
            model=self.config.CHAT_MODEL,
            messages=[
                {
                    "role":"user",
                    "content":[
                        {
                            "type": "text",
                            "text": f"""
                            次のテキストは質問に理由をつけて答えてもらったものである。
                            これをもとに質問に答えを単語のみ答えよ。複数ある場合は複数の単語で答えよ。
                            情報が不足している場合には「分かりません」と答えよ。
                            質問：{question}
                            理由と答え：{process_answer}
                            """
                        }
                    ]
                }
            ],
            max_tokens=max_tokens,
            temperature=0,        
        )
        answer = response.choices[0].message.content
        if answer is None or answer.strip() == "":
            answer = "分かりません"
        answer = answer.strip()
        return answer

In [ ]:
query_df

In [ ]:
rag_generator = RAG_Generater(client, config, index)
pred_answers = []
for n, row in query_df.iterrows():
    question = row.problem
    rag_indices = rag_generator.extract_rag_indices(question)
    for i, row2 in rag_df.iloc[rag_indices].iterrows():
        encode_image, search = row2.image, row2.search
        process_answer = rag_generator.generate_process_answer(question, text=search)
        print(n, question, row2.pdf_path, row2.page)
        display(Markdown(process_answer))
        if "分かりません" not in process_answer:
            final_answer = rag_generator.generate_answer(question, process_answer, max_tokens=54)
            pred_answers.append([row2.pdf_path, row2.page, encode_image, search, process_answer, final_answer])       
            break
    else:
        pred_answers.append([row2.pdf_path, row2.page, encode_image, search, process_answer, "分かりません"]) 
query_df[["pdf_path", "page", "encode_image", "text", "pred_process_answer", "pred_final_answer"]] = pred_answers
query_df["pred_final_answer"] = query_df["pred_final_answer"].str.replace("\n"," ")


In [ ]:
for _,row in query_df.iterrows():
    print(_,row.problem)
    display(Markdown(row.pred_final_answer)) 

## 出力

In [ ]:
output_csv_path = f"{base_path}/output/predictions.csv"
output_zip_path = f"{base_path}/output/predictions.zip"
query_df[["index", "pred_final_answer"]].to_csv(output_csv_path, index=False, header=False)

with zipfile.ZipFile(
        output_zip_path, 'w',
        compression=zipfile.ZIP_DEFLATED,
        compresslevel=9) as zf:
    zf.write(output_csv_path, arcname=os.path.basename(output_csv_path))